텐서플로우 2.0 기반 huggingface 라이브러리
KOBERT를 활용한 네이버쇼핑 감성분석

huggingface 패키지를 Colab에 설치

In [1]:
!pip install transformers==2.10

     |████████████████████████████████| 665kB 6.1MB/s 
     |████████████████████████████████| 1.2MB 8.8MB/s 
     |████████████████████████████████| 5.6MB 18.6MB/s 
     |████████████████████████████████| 870kB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d15f841f0d69d35a5cbbc8fd0ac443ed581ebbbfe9c707dfec83b45ffdb648b0
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
!pip install tensorflow-addons

     |████████████████████████████████| 706kB 4.4MB/s 


In [3]:
!pip install -q pyyaml h5py

# KOBERT 활용한 인스타그램 광고분류 모델

#목차
<b>1) 인스타그램 텍스트 데이터 불러오기 및 전처리  
2) BERT 인풋에 맞게 변환  
3) 버트를 활용한 감성분석 모델 생성  
4) 훈련 및 성능 검증  
5) 실제 데이터 적용</b>

텐서플로우 2와 필요한 모듈들을 임포트



In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

구글 드라이브와 Colab을 연동하기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/DataScience/data/instagram

/content/drive/MyDrive/DataScience/data/instagram


### 데이터 불러오기

In [7]:
# 인스타그램 데이터 불러오기
train = pd.read_csv("train_final.csv")
test_ = pd.read_csv("test_final.csv")

train.drop('Unnamed: 0', axis=1, inplace=True)
test_.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
# train.columns = ['document', 'label']
# test_.columns = ['document', 'label']

In [9]:
train

,text,label
0,편안해보이는 영상의 원본은 집안을 울리는 공사소리 얼마전 삐그덕대던 보일러가 드디...,1
1,광고아님 제품협찬 이지만 노출의무 없음 찐사용 욕실 3총사 요즘 씻고 건조해서 욕...,1
2,다들 날씨도 좋은데 파출소포차 하세요 무야호 dantbar1987,0
3,사진맛집 브런치맛집 아날로그 포레스트 analog forest 인천사시는분들 꼭가...,1
4,몇개월전 이미 아이언은 바꿨지만 이제 올리네 아버지채 받아서 쓰다가 얼마전 태일러...,1
...,...,...
56857,윤조에센스 120ml 사전예약주문 1년에 한번 나와요 설화수 윤조에센스 윤조 윤조...,1
56858,NEW 자음생앰플 인삼에서 찾은 빠르고 강력한 고기능 안티에이징 성분 귀하게 얻은...,1
56859,콜라겐 전 마셔요 송혜교가 선택한 마시는 프리미엄 콜라겐 슈퍼콜라겐 에센스를 공개...,1
56860,블로그로 문의주신 고객님 주말지내고 바로 원주로 달려간 날 R사 화장품 사용으로 ...,1


In [10]:
test_

,text,label
0,181125 이히히히힣힠 대놓고 자랑중입니다 프라엘 남편최고,0
1,구서방 자꾸왜이래 고맙게시리 피부나이 20대로 살고싶은 내나이 30대중반 관리잘하...,0
2,하루 9분투자를 못해 방치만 하고 있네 오늘부터 부지런히 사용해보자 프라엘 프라엘...,0
3,센스쟁이 화이트데이 육아 소통 일상 셀카 셀피 셀스타그램 경산 구미 대구맘 대구 ...,0
4,LG 프라엘 두부처럼 모태미녀가 아니라서 중력을 거스르려는 헛된 욕심 효과를 바래...,0
...,...,...
25743,kyo1122 코로나 빨리 사라져서 여행 가고 싶다 이런 공항kyo 스타일 좋아 ...,0
25744,수분을 가득 머금은 거품이 피부 노폐물을 깔끔히 없애고 피부의 촉촉함은 유지시켜주...,0
25745,kyo1122 좋아하는 모연 장면 낮술 좋네 였던가 기억이 가물 credit to...,0
25746,kyo1122 귀여운 표정의 혜교님 보니 기분이 좋아질려고 합니다 credit t...,0


In [11]:
# 타겟레이블 분포 확인 타겟레이블 분포 확인
print(train['label'].value_counts(normalize=True))
print(test_['label'].value_counts(normalize=True))

1    0.597306
0    0.402694
Name: label, dtype: float64
1    0.556199
0    0.443801
Name: label, dtype: float64


In [12]:
# 훈련셋과 검증셋으로 분리
val, test = train_test_split(test_,
                               random_state=3, 
                               test_size=0.5,
                               stratify=test_['label'])

print(f"validation : {val.shape}\ntest : {test.shape}")

validation : (12874, 2)
test : (12874, 2)


In [13]:
val.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [14]:
# 데이터별 타겟 분포 확인
print(f"학습용데이터\n{train['label'].value_counts(normalize=True)}")
print(f"테스트용데이터\n{test['label'].value_counts(normalize=True)}")

학습용데이터
1    0.597306
0    0.402694
Name: label, dtype: float64
테스트용데이터
1    0.55616
0    0.44384
Name: label, dtype: float64


In [15]:
# 훈련셋의 앞부분 출력
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.head(10)

,text,label
0,편안해보이는 영상의 원본은 집안을 울리는 공사소리 얼마전 삐그덕대던 보일러가 드디...,1
1,광고아님 제품협찬 이지만 노출의무 없음 찐사용 욕실 3총사 요즘 씻고 건조해서 욕...,1
2,다들 날씨도 좋은데 파출소포차 하세요 무야호 dantbar1987,0
3,사진맛집 브런치맛집 아날로그 포레스트 analog forest 인천사시는분들 꼭가...,1
4,몇개월전 이미 아이언은 바꿨지만 이제 올리네 아버지채 받아서 쓰다가 얼마전 태일러...,1
5,랑콩뜨레 딸기몽블랑 현대백화점 광고아님 화이트 데이 인님들은 어떻게 보내고 계신가...,1
6,oldboy 스끼야끼 부산맛집 서면맛집 스끼야끼맛집 광고아님 찐맛집 뜨끈한게 생각...,1
7,외출 불광천에 오리랑 멍멍이들 보고 좋아하는줄 알았더니 스타벅스 가 너의 시선을 ...,1
8,넘나 해보고팠던 애쉬바이올렛 하나하나 해보고 싶었던 일들을 하고 있는 요즘 탈색 ...,1
9,21 3 13 주말 나들이 세번째 방문인데 후 정말 이사하고 싶은 맛이닿ㅎㅎ 이번...,1


## 버트 인풋 만들기

한국어 데이터로 훈련되었고, SKT에서 만든 KoBERT를 사용  

https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드 가져옴

In [16]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

kobert 토크나이즈를 임포트

In [17]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌  
tokenizer.tokenize => 문장을 토큰화

In [18]:
print(tokenizer.encode("너무 맛있어서 다시 주문했어요"))

[2, 1458, 1967, 7141, 6855, 6553, 1574, 4227, 7874, 3]


In [19]:
print(tokenizer.tokenize("너무 맛있어서 다시 주문했어요"))

['▁너무', '▁맛', '있', '어', '서', '▁다시', '▁주문', '했어요']


In [20]:
print(tokenizer.encode("너무맛있어서다시주문했어요"))

[2, 1458, 6164, 7141, 6855, 6553, 5782, 6705, 7276, 6234, 7874, 3]


네이버 쇼핑 리뷰 문장들을 버트 인풋으로 변환  
문장은 토큰 인풋, 세그먼트 인풋, 마스크 인풋으로 변환  
huggingface에서의 순서 [토큰 인풋, 마스크 인풋, 세그먼트 인풋]

In [21]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 64
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "text"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

100%|██████████| 56862/56862 [00:39<00:00, 1425.85it/s]


In [22]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
val_x, val_y = load_data(val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 12874/12874 [00:08<00:00, 1454.87it/s]


## 버트를 활용한 감성분석 모델 만들기

In [23]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [24]:
bert_outputs

(<KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'tf_bert_model')>,
 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)

In [25]:
bert_outputs = bert_outputs[1]

In [26]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

In [27]:
ads_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
ads_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(ads_drop)
ads_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], ads_first)
ads_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [28]:
ads_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 64, 768), (N 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]            

## 훈련 및 성능 검증

모델 학습

In [ ]:
ads_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=64, validation_data=(val_x, val_y))

Epoch 1/4
889/889 [==============================] - 874s 926ms/step - loss: 0.5834 - accuracy: 0.6801 - val_loss: 0.6044 - val_accuracy: 0.6931
Epoch 2/4
889/889 [==============================] - 822s 925ms/step - loss: 0.2992 - accuracy: 0.8480 - val_loss: 0.7908 - val_accuracy: 0.6926
Epoch 3/4
889/889 [==============================] - 821s 924ms/step - loss: 0.2483 - accuracy: 0.8785 - val_loss: 1.1193 - val_accuracy: 0.6849
Epoch 4/4
889/889 [==============================] - 821s 924ms/step - loss: 0.1991 - accuracy: 0.9095 - val_loss: 1.2068 - val_accuracy: 0.6990


모델 저장

In [ ]:
# ads_model.save('/content/gdrive/MyDrive/DataScience/model/mymodel')

In [ ]:
# ads_model.save_weights('/content/drive/MyDrive/DataScience/model/saveweights/ads_save.h5')

In [29]:
ads_model.load_weights('/content/drive/MyDrive/DataScience/model/saveweights/ads_save.h5')

훈련 모델의 예측 성능을 F1 SCORE로 체크하기 위한 작업

In [30]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [31]:
test_set = predict_load_data(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 12874/12874 [00:08<00:00, 1545.46it/s]


In [32]:
test_set

[array([[   2, 1209, 7681, ..., 5742, 7946,    3],
        [   2,  668, 7214, ..., 5592,  761,    3],
        [   2, 4092, 7342, ..., 6588,  882,    3],
        ...,
        [   2, 4102,  777, ...,    1,    1,    1],
        [   2,  517,  375, ..., 3138, 3647,    3],
        [   2,  695,  357, ...,  409,  517,    3]]),
 array([[1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [33]:
preds = ads_model.predict(test_set)

In [34]:
# 부정이면 0, 긍정이면 1 출력
preds

array([[7.2416019e-01],
       [9.9883217e-01],
       [9.9889225e-01],
       ...,
       [8.8324869e-04],
       [8.4828728e-01],
       [9.9853039e-01]], dtype=float32)

우리가 훈련한 모델을 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [35]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.72      0.51      0.60      5714
           1       0.68      0.84      0.75      7160

    accuracy                           0.69     12874
   macro avg       0.70      0.68      0.68     12874
weighted avg       0.70      0.69      0.68     12874



In [36]:
import logging
tf.get_logger().setLevel(logging.ERROR)

# 실제 데이터 적용

In [39]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = ads_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      return "(부정 확률 : %.2f) 광고가 아닙니다." % (1-predict_value)
    elif predict_answer == 1:
      return "(긍정 확률 : %.2f) 광고입니다." % predict_value

In [40]:
review_evaluation_predict("두번 다시 안 살꺼에요")

'(부정 확률 : 1.00) 광고가 아닙니다.'

In [41]:
review_evaluation_predict("배송이 좀 느리긴 했어도 품질 자체는 믿을만해요")

'(부정 확률 : 1.00) 광고가 아닙니다.'

In [42]:
review_evaluation_predict("디자인이 매우 세련되었으나 쉽게 망가지네요. 내구도가 엉망이에요")

'(부정 확률 : 1.00) 광고가 아닙니다.'

### 에어팟 리뷰데이터 분류

In [ ]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      return "(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value)
    elif predict_answer == 1:
      return "(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value

In [43]:
test['evaluation'] = test['text'].apply(review_evaluation_predict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
test

,text,label,evaluation
0,그리팅바이탈식단 맛도좋고 영양도 좋고 건강에는 더욱더 좋고 가정주부에겐 너무나 편한...,1,(긍정 확률 : 0.72) 광고입니다.
1,LG전자초프리미엄샵오픈 기다리고 기다리던 롯데백화점 중동점LG 전자 초프리미엄샵 ...,1,(긍정 확률 : 1.00) 광고입니다.
2,정직한 판매 믿고 사는 코스메틱 뷰티몰 화장품 건강기능식품 100 본사 상품유통기한...,1,(긍정 확률 : 1.00) 광고입니다.
3,평소 윤조에센스 쓰는데 5세대가 나왔다길래 체험단 신청했다가 덜컥 당첨되었다 큰 게...,0,(부정 확률 : 0.87) 광고가 아닙니다.
4,클스마스 잘 보내고 있나요 전 어제 쫌 무리했나봐요 아침에 라면 하나먹고 멍 때리고...,0,(부정 확률 : 0.92) 광고가 아닙니다.
...,...,...,...
12869,내몸이 천냥이면 눈은 구백냥 제2의 눈 선글라스 트루아이 트루아이 의사 선생님도 인...,1,(긍정 확률 : 0.60) 광고입니다.
12870,예뻐지자 프라엘,0,(부정 확률 : 1.00) 광고가 아닙니다.
12871,정말 간만에 프라엘 결혼후 관리 다시시작,0,(부정 확률 : 1.00) 광고가 아닙니다.
12872,artflora 후기영상 자이언트카네이션 선물포장 메이크업 선생님께 스승의날 선물...,1,(긍정 확률 : 0.85) 광고입니다.


In [48]:
# test.to_csv('ads_output.csv', encoding='utf-8-sig', mode='w')